In [10]:
import sys
import os

In [11]:
print(os.getcwd())

/home/gqxwolf/mydata/GNN_Combinatroal/skylineGNN/003


In [12]:
import torch

In [13]:
print(torch.__version__)
print(torch.version.cuda)

1.9.0
11.1


In [14]:
sys.path.append('/home/gqxwolf/mydata/GNN_Combinatroal/skylineGNN/003')
sys.path.append('/home/gqxwolf/mydata/GNN_Combinatroal/skylineGNN/utilities')
print(sys.path)

['/home/gqxwolf/mydata/GNN_Combinatroal/skylineGNN/003', '/home/gqxwolf/.vscode/extensions/ms-toolsai.jupyter-2021.8.2041215044/pythonFiles', '/home/gqxwolf/.vscode/extensions/ms-toolsai.jupyter-2021.8.2041215044/pythonFiles/lib/python', '/home/gqxwolf/anaconda3/envs/skylineGNN/lib/python38.zip', '/home/gqxwolf/anaconda3/envs/skylineGNN/lib/python3.8', '/home/gqxwolf/anaconda3/envs/skylineGNN/lib/python3.8/lib-dynload', '', '/home/gqxwolf/.local/lib/python3.8/site-packages', '/home/gqxwolf/anaconda3/envs/skylineGNN/lib/python3.8/site-packages', '/home/gqxwolf/anaconda3/envs/skylineGNN/lib/python3.8/site-packages/IPython/extensions', '/home/gqxwolf/.ipython', '/home/gqxwolf/mydata/GNN_Combinatroal/skylineGNN/003', '/home/gqxwolf/mydata/GNN_Combinatroal/skylineGNN/utilities', '/home/gqxwolf/mydata/GNN_Combinatroal/skylineGNN/003', '/home/gqxwolf/mydata/GNN_Combinatroal/skylineGNN/utilities']


In [15]:
from torch.nn import CrossEntropyLoss, BCELoss, BCEWithLogitsLoss
from torch_geometric.data import DataLoader
# from torch_geometric.loader import DataLoader
import numpy as np
from model import GraphEncoder
from readData import LoadData, LoadGraph
import os
from MultiCostNetworks import MultiCostNetworks
import torch.nn.functional as F
from drawRawGraphs import draw_graph
import time
from torch_geometric.utils import subgraph, k_hop_subgraph
import math

In [16]:
from readCheckPoints import getMaxCheckPointName
from MultiCostNetworks import MultiCostNetworks
from readData import LoadData, LoadGraph
from model import GraphEncoder
from drawRawGraphs import draw_graph_with_DataObj

In [17]:
path = "/home/gqxwolf/mydata/projectData/skylineGNN/dataset/C9_NY_NONE/C9_NY_NONE_30K/processed/level8"
train_paths_folder = "/home/gqxwolf/mydata/projectData/skylineGNN/results_back/C9_NY_NONE_30K/processed/level8/results"
check_point_file = '/home/gqxwolf/mydata/projectData/skylineGNN/C9_NY_NONE_30K/processed/level8/512_512_64_200_1_Embed-1_5787_train_checkpoint_GCN_ConLoss-2_2GNN3FF_10k_train_samples.pt'
data_folder = "/home/gqxwolf/mydata/projectData/skylineGNN/processed/C9_NY_NONE_30K/processed/level8"

print(path)
print(train_paths_folder)
print(check_point_file)
print(data_folder)

/home/gqxwolf/mydata/projectData/skylineGNN/dataset/C9_NY_NONE/C9_NY_NONE_30K/processed/level8
/home/gqxwolf/mydata/projectData/skylineGNN/results_back/C9_NY_NONE_30K/processed/level8/results
/home/gqxwolf/mydata/projectData/skylineGNN/C9_NY_NONE_30K/processed/level8/512_512_64_200_1_Embed-1_5787_train_checkpoint_GCN_ConLoss-2_2GNN3FF_10k_train_samples.pt
/home/gqxwolf/mydata/projectData/skylineGNN/processed/C9_NY_NONE_30K/processed/level8


In [19]:
node_dim = 2
edge_dim = 3
embedding_dim = 512
hidden_dim = 512
batch_size = 64
enable_embed = True
output_dim = 3
model_name = "GCN"
heads = 2
conn_loss_enable = False
enable_edge_attr= True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

num_node, graph = LoadGraph(path, logger=None, normalization_node=False, normalization_edge=True)

print("==========================================")

train_dataset = MultiCostNetworks(
    graph, train_paths_folder=train_paths_folder, root=data_folder, split='train')
test_dataset = MultiCostNetworks(
    graph, train_paths_folder=train_paths_folder, root=data_folder, split='test')
train_loader = DataLoader(train_dataset, drop_last=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, drop_last=True, batch_size=batch_size)

cuda
coordinates with raw values:---------------------------------------
   node_id            lattitude           longitude
0        0   0.8193483335404892  0.8074799968985644
1        1   0.8180258014613173   0.807905195678984
2        2  0.43024765535434994  0.7378674530343119
3        3    0.652546996505464  0.9097328000840434
4        4   0.6543909446263001  0.9090074609879935
normalized coordinates:---------------------------------------
   node_id            lattitude           longitude
0        0   0.8193483335404892  0.8074799968985644
1        1   0.8180258014613173   0.807905195678984
2        2  0.43024765535434994  0.7378674530343119
3        3    0.652546996505464  0.9097328000840434
4        4   0.6543909446263001  0.9090074609879935
0.966197737537783 0.038329258691419454
0.9460047572239887 0.007753624819599736
edge attributes with raw values:---------------------------------------
   src_node_id  dest_node_id   cost1 cost2 cost3
0         3206          3207   752.0  75